# Spotify Web API

In [1]:
import json
from yaml import safe_load, dump
from typing import Final
from spotipy import Spotify
from spotipy.oauth2 import SpotifyOAuth
from functools import lru_cache

In [2]:
import logging

logging.getLogger(__name__)
logging.getLogger('spotipy').setLevel(logging.DEBUG)
logging.basicConfig(level=logging.DEBUG, filename='logs/playground.log', filemode='w')

## Read client data from conf.yml file

In [3]:
def read_client_data() -> dict[str, str]:
    with open('conf.yml', 'r') as f:
        data = safe_load(f)
        return data
    
client_data: dict[str, str] = read_client_data()
CLIENT_ID: Final[str] = client_data.get('client_id')
CLIENT_SECRET: Final[str] = client_data.get('client_secret')
REDIRECT_URI: Final[str] = client_data.get('redirect_uri')

## Create Spotify object with client authorization

In [4]:
# Authenticate with Spotify using OAuth
scopes="user-read-email,user-library-read,user-top-read,user-read-recently-played"
auth_manager=SpotifyOAuth(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI, scope=scopes)
sp = Spotify(auth_manager=auth_manager)

## Get User

In [5]:
sp.me()

{'country': 'IL',
 'display_name': 'Idan Haviv',
 'email': 'haviv1idan@gmail.com',
 'explicit_content': {'filter_enabled': False, 'filter_locked': False},
 'external_urls': {'spotify': 'https://open.spotify.com/user/haviv1idan'},
 'followers': {'href': None, 'total': 4},
 'href': 'https://api.spotify.com/v1/users/haviv1idan',
 'id': 'haviv1idan',
 'images': [{'height': 300,
   'url': 'https://platform-lookaside.fbsbx.com/platform/profilepic/?asid=1520398911315405&height=300&width=300&ext=1732865043&hash=AbZeSJxa4eQk5-KWKi762vBJ',
   'width': 300},
  {'height': 64,
   'url': 'https://platform-lookaside.fbsbx.com/platform/profilepic/?asid=1520398911315405&height=50&width=50&ext=1732865043&hash=AbaRWgRCQVG-fLjNZGLp79Vc',
   'width': 64}],
 'product': 'premium',
 'type': 'user',
 'uri': 'spotify:user:haviv1idan'}

## Top Tracks

### Get current user top tracks in the short term, medium term and long term.

In [4]:
# Short term
top_tracks = sp.current_user_top_tracks(time_range='short_term', limit=50)

In [ ]:
# Medium term
top_tracks = sp.current_user_top_tracks(time_range='medium_term', limit=50)

In [ ]:
# Long term
top_tracks = sp.current_user_top_tracks(time_range='long_term', limit=50)

### Get list of top tracks and the artists

In [ ]:
list(map(lambda x: f"{x['artists'][0]['name']} - {x['name']}", top_tracks['items']))

### analyze the top tracks and return a dict of each artist and his tracks

In [9]:
# artists <-> tracks analyze with artist is the most accured in user top tracks
from copy import deepcopy
def artists_tracks_analyze(top_tracks: dict) -> dict[str, str]:
    tracks = deepcopy(top_tracks)
    artists_tracks = {}
    for i, item in enumerate(tracks['items']):
        print(f"{i}. {item['name']} - {item['artists'][0]['name']}")
        artist_name = item['artists'][0]['name']
        track_name = item['name']
        if artist_name not in artists_tracks.keys():
            artists_tracks[artist_name] = [track_name]
        else:
            artists_tracks[artist_name].append(track_name)

    return artists_tracks
     

In [ ]:
artists_tracks = artists_tracks_analyze(top_tracks)

In [ ]:
artists_tracks

### Count for each artist how muck songs he have in user top tracks in given term

In [ ]:
for artist, tracks in artists_tracks.items():
    print(f"{artist} , {len(tracks)}")

## Saved Tracks

### Get all saved tracks of current user

In [4]:
# Wrapper function for saved tracks to reduce api calls
@lru_cache(maxsize=1)
def get_saved_tracks():
    saved_tracks = []
    tracks = sp.current_user_saved_tracks()
    while tracks:
        for i, item in enumerate(tracks['items']):
           saved_tracks.append(item)
           track = item['track']
           if i % 10 == 0:
               print(i, track['artists'][0]['name'], ' - ', track['name'], end="\n")
           else:
              print(i, track['artists'][0]['name'], ' - ', track['name'], end=" ")
        
        if tracks['next']:
            tracks = sp.next(tracks)
        else:
            tracks = None
    return saved_tracks


In [ ]:
saved_tracks = get_saved_tracks()

In [ ]:
for track in saved_tracks:
    print(track['track']['name'], end="\n")

In [49]:
top_tracks = sp.current_user_top_tracks(time_range='short_term')

In [ ]:
for item in top_tracks['items']:
    print(json.dumps(item))

### Read user saved tracks and store to yaml file

In [19]:
saved_tracks = sp.current_user_saved_tracks(10)
saved_tracks = saved_tracks['items']

In [ ]:
saved_tracks_dict = {}
for track in saved_tracks:
    track_id = track['track']['id']
    obj_track = {
        'added_at': track['added_at'],
        'name': track['track']['name'],
        'artist': track['track']['artists'][0]['name'],
        'popularity': track['track']['popularity'],
        'uri': track['track']['uri']
    }
    saved_tracks_dict[track_id] = obj_track

print(saved_tracks_dict)
with open('test.json', 'w') as f:
    json.dump(saved_tracks_dict, f)

    

## Recently Played Tracks

In [13]:
result = sp.current_user_recently_played(10)

In [ ]:
result['items']
for item in result['items']:
    played_at = item['played_at']
    track = item['track']
    track_name = track['name']
    artist = track['artists'][0]['name']
    print(f"Song: {track_name}, artist: {artist} was played at: {played_at}")

In [12]:
# Wrapper function to get recetly played tracks
@lru_cache(maxsize=1)
def get_recently_played(limit: int = 50, before: str = None, after: str = None):
    recently_played = []
    tracks = sp.current_user_recently_played(limit=limit, after=after, before=before)
    while tracks:
        for i, item in enumerate(tracks['items']):
           recently_played.append(item)
           track = item['track']
           if i % 10 == 0:
               print(i, track['artists'][0]['name'], ' - ', track['name'], end="\n")
           else:
              print(i, track['artists'][0]['name'], ' - ', track['name'], end=" ")
        
        if tracks['next']:
            tracks = sp.next(tracks)
        else:
            tracks = None
    return recently_played

## Top Artists

### Get current user top 10 artists

In [ ]:
ranges = ['short_term', 'medium_term', 'long_term']

for range in ranges:
    print(f'{range= }')

    top_artists = sp.current_user_top_artists(time_range=range, limit=10)

    for i, item in enumerate(top_artists['items']):
        print(f"{i}. {item['name']}")
    print()

# Get Recommendations

In [6]:
from random import shuffle

# Get the user's top tracks
top_tracks = sp.current_user_top_tracks(limit=10, time_range='long_term')

# Get the user's liked tracks
liked_tracks = sp.current_user_saved_tracks(limit=10)

# Get the user's listening history
recent_tracks = sp.current_user_recently_played(limit=10)

# Extract a list of the top track IDs
top_track_ids = list(map(lambda track: track['id'], top_tracks['items']))
# top_track_ids = [track['id'] for track in top_tracks['items']]

# Extract a list of the liked track IDs
liked_track_ids = list(map(lambda track: track['track']['id'], liked_tracks['items']))
# liked_track_ids = [track['track']['id'] for track in liked_tracks['items']]

# Extract a list of the history track IDs
history_track_ids = list(map(lambda track: track['track']['id'], recent_tracks['items']))
# history_track_ids = [track['track']['id'] for track in recent_tracks['items']]

# Combine the three lists and shuffle them randomly
seed_track_ids = top_track_ids + liked_track_ids + history_track_ids
shuffle(seed_track_ids)

recommendations = sp.recommendations(
    seed_tracks=seed_track_ids[:5]
)

In [ ]:
recommendations

In [8]:
from prettytable import PrettyTable

table = PrettyTable(
    field_names=['name', 'popularity', 'artist', 'album']
)

for track in recommendations['tracks']:
    _name = track['name']
    _popularity = track['popularity']
    _artist = track['artists'][0]['name']
    _album = track['album']['name']
    table.add_row([_name, _popularity, _artist, _album])
    # print(f"track: {_name}\t popularity: {_popularity}\t artist: {_artist}\t album: {_album}")

print(table)

+------------------------------------------+------------+-----------------------------+-----------------------------------------+
|                   name                   | popularity |            artist           |                  album                  |
+------------------------------------------+------------+-----------------------------+-----------------------------------------+
|          Wild Ones (feat. Sia)           |     78     |           Flo Rida          |                Wild Ones                |
|     Mambo No. 5 (a Little Bit of...)     |     75     |           Lou Bega          |          A Little Bit of Mambo          |
|                  Cancer                  |     58     |      Twenty One Pilots      |                  Cancer                 |
|                 Fat Lip                  |     72     |            Sum 41           |          All Killer, No Filler          |
|     There's Nothing Holdin' Me Back      |     84     |         Shawn Mendes        |   